In [1]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import TranslationPipeline

from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [48]:
df_train = pd.read_csv('../../../data/frameworks_data/data_v0.4.4/data_v0.4.4_train.csv')
df_test = pd.read_csv('../../../data/frameworks_data/data_v0.4.4/data_v0.4.4_test.csv')
df_val = pd.read_csv('../../../data/frameworks_data/data_v0.4.4/data_v0.4.4_val.csv')

df_tot = pd.concat([df_train, df_test, df_val])

In [50]:
df_tot[['excerpt', 'entry_id', 'language', 'sectors', 'subpillars']]\
            .to_csv('../../../data/frameworks_data/data_v0.4.4/data_v0.4.4_full.csv')

name: English     code: en       confidence:  87.0 read bytes:  1154


In [51]:
df_tot = pd.read_csv('../../../data/frameworks_data/data_v0.4.4/data_v0.4.4_full.csv', index_col=0)
df_specific_needs = pd.read_csv('../../../data/secondary_tags/specific_needs_groups_final.csv', index_col=0)

In [52]:
df_tot.head(2)

,excerpt,entry_id,language,sectors,subpillars
0,Market monitoring by the World Food Programme ...,163664,en,['Food Security'],['Impact->Impact On Systems And Services']
1,Quarantine Facilities: ninety-three shelters i...,162812,en,['Health'],['Capacities & Response->International Response']


In [54]:
df_specific_needs.head(2)

,excerpt,entry_id,specific_needs_groups
389,"In Syria, UNHCR provided guidance to partners ...",133321,['Persons with Disability']
632,HelpAge’s country team and partners are reachi...,25684,['Persons with Disability']


In [49]:
len(set(list(df_specific_needs.entry_id)).intersection(set(list(df_tot.entry_id))))

6956

In [56]:
merged = pd.merge(
    df_tot[['entry_id', 'language']],
    df_specific_needs,
    on='entry_id',
    how='right'
)

In [72]:
from langdetect import detect
tmp = []
for text in tqdm(not_labeled_data.excerpt):
    lang = detect(text)
    tmp.append(lang)


100%|██████████| 584/584 [00:05<00:00, 116.63it/s]


In [76]:
merged.to_csv('../../../data/secondary_tags/specific_needs_groups_final_with_language.csv')

In [2]:
df_subs = pd.read_csv('../../../data/secondary_tags/specific_needs_groups_final_with_language.csv')

In [3]:
en_df = df_subs[df_subs.language=='en']
fr_df = df_subs[df_subs.language=='fr']
es_df = df_subs[df_subs.language=='es']

In [81]:
"""model = AutoModelWithLMHead.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
translation = TranslationPipeline(model=model, tokenizer=tokenizer)

fr_to_en_texts = []
false_inds = []
for text in tqdm(fr_df.excerpt):
    try:
        translated_text = translation(text, max_length=512)[0]['translation_text']
        fr_to_en_texts.append(translated_text)
    except IndexError :
        false_inds.append(text)
        pass"""

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/301M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/802k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

 45%|████▌     | 569/1255 [19:15<26:46,  2.34s/it]  Token indices sequence length is longer than the specified maximum sequence length for this model (1486 > 512). Running this sequence through the model will result in indexing errors
Your input_length: 1486 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
100%|██████████| 1255/1255 [47:58<00:00,  2.29s/it] 


In [84]:
"""fr_df = fr_df.drop(fr_df.index[[569]])"""

In [87]:
"""en_from_fr = fr_df.copy()
en_from_fr['excerpt'] = fr_to_en_texts
en_from_fr.to_csv('../../../data/secondary_tags/augmented_data/en_from_fr.csv')
"""

In [4]:
#en_from_fr

In [5]:
es_df = es_df.iloc[532:]

In [ ]:
model = AutoModelWithLMHead.from_pretrained("Helsinki-NLP/opus-mt-es-fr")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-fr")
translation = TranslationPipeline(model=model, tokenizer=tokenizer)

es_to_fr_texts = []
false_inds = []
excerpt_list = list(es_df.excerpt)
for i in tqdm(range(len(excerpt_list))):
    try:
        translated_text = translation(excerpt_list[i], max_length=512)[0]['translation_text']
        es_to_fr_texts.append(translated_text)
    except IndexError :
        false_inds.append(i)
        pass
    


 17%|█▋        | 310/1820 [16:42<42:30,  1.69s/it]  Token indices sequence length is longer than the specified maximum sequence length for this model (1381 > 512). Running this sequence through the model will result in indexing errors
Your input_length: 1381 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 533 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
 44%|████▍     | 800/1820 [49:14<1:10:02,  4.12s/it]Your input_length: 618 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 511 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
 92%|█████████▏| 1673/1820 [1:49:21<09:20,  3.81s/it]  

In [91]:
if len (false_inds)>0:
    es_df = es_df.drop(es_df.index[false_inds])
    


In [94]:
es_df = es_df.iloc[:531]

In [95]:
fr_from_es = es_df.copy()
fr_from_es['excerpt'] = es_to_fr_texts
fr_from_es.to_csv('../../../data/secondary_tags/augmented_data/fr_from_es_first_part.csv')

In [65]:
not_labeled_data = merged[merged.language.isna()]
not_labeled_data

,entry_id,language,excerpt,specific_needs_groups
16,106263,NaN,Entre los perfiles se identifican mayormente h...,['Pregnant or Lactating Women']
26,100912,NaN,"On March 18, Brazil ordered a partial closing ...",['Unregistered Refugee']
43,27456,NaN,"The Government has identified nearly 28,000 vu...","['Persons with Disability', 'Female Head of Ho..."
55,39418,NaN,Since the start of the evacuation operation fr...,['Unaccompanied or Separated Children']
63,218251,NaN,"[1 Oct 2020, Whole of Syria] Over a third of t...",['Persons with Disability']
...,...,...,...,...
7241,287561,NaN,The Nutrition Cluster urges all parties to the...,['Pregnant or Lactating Women']
7280,236213,NaN,An estimated 80 per cent of girls with disabil...,['Persons with Disability']
7386,322787,NaN,The initial consignment of vaccines received o...,['Chronically Ill']
7424,311810,NaN,Approximately 15 per cent of Sudan’s populatio...,['Persons with Disability']


In [33]:
languages = []
for text_tmp in tqdm(df_full.excerpt):
    try:
        lg = Detector(text_tmp).languages[0]

        languages.append(lg.name)
    except:
        languages.append('UNKNOWN')

  0%|          | 0/7540 [00:00<?, ?it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
 12%|█▏        | 921/7540 [00:00<00:00, 9204.37it/s]Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language 

In [34]:
df_full['language'] = pd.Series(languages)

In [ ]:
def keep_wanted_languages(x):
    if x=='English' or x=='French'

df_full = df_full[df_full.language.apply()]

In [36]:
#df_full.to_csv('../../../data/frameworks_data/data_v0.5.1/data_v0.5.1_full.csv')

In [ ]:
fr_df = df_full[df_full.language=='French']
fr_df

In [25]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-small')

model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
fr_to_en_t5 = []
for text in tqdm(fr_df.excerpt):
    input_ids = tokenizer("translate French to English: "+text, return_tensors="pt").input_ids  # Batch size 1

    outputs = model.generate(input_ids)

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    fr_to_en_t5.append(decoded)

In [64]:
languages = []
for text_tmp in tqdm(not_labeled_data.excerpt):
    b = TextBlob(text_tmp)
    languages.append(b.detect_language())

 36%|███▋      | 213/584 [01:24<02:26,  2.53it/s]


HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
np.unique(languages)

In [6]:
en_df = df_subs[df_subs.language=='en']
fr_df = df_subs[df_subs.language=='fr']
es_df = df_subs[df_subs.language=='es']

In [ ]:
"""
1) code to

fr -> es
es -> fr
en -> es
en -> fr
fr -> en
es -> en

2) detect language in text and apply language

3) Merge models: demographic groups, sectors, 2d subpillars

4) Augment data for 1d subpillars, specific needs groups
"""

In [9]:
model = AutoModelWithLMHead.from_pretrained("Helsinki-NLP/opus-mt-fr-es")
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-es")

/home/selim/anaconda3/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:843: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/332M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/812k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/819k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

In [10]:
translation = TranslationPipeline(model=model, tokenizer=tokenizer)

In [11]:
fr_to_es_texts = []
for text in tqdm(fr_df.excerpt.sample(n=50)):
    translated_text = translation(text, max_length=256)[0]['translation_text']
    fr_to_es_texts.append(translated_text)

 48%|████▊     | 24/50 [01:07<01:13,  2.83s/it]


KeyboardInterrupt: 